# url 주소 획득

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
# driver_path = '/path/to/chromedriver'
driver = webdriver.Chrome()
url = "https://www.visitjeju.net/kr/detail/list?menuId=DOM_000001707000000000&cate1cd=cate0000000004#p1&pageSize=1004&sortListType=reviewcnt&viewType=map&isShowBtag&tag"
driver.get(url)
# 페이지 로드를 기다립니다.
driver.implicitly_wait(10)
s_tit_list = []
href_list = []
# 첫 번째 페이지에서 각 관광지 정보를 클릭합니다.
for i in tqdm(range(1, 1005)):  # 숙박업소 수
    try:
        element = driver.find_element(By.CSS_SELECTOR, f"#map_outline > div > ul > li:nth-child({i}) > dl > dt > a > p.s_tit")
        s_tit = element.text
        # element가 화면 내에 보이도록 스크롤
        #driver.execute_script("arguments[0].scrollIntoView();", element)
        # element를 클릭하기 전에 충분히 대기
        ActionChains(driver).move_to_element(element).perform()
        time.sleep(2)
        # element 클릭
        driver.execute_script("arguments[0].click();", element)
        # 'link1' 클래스를 가진 모든 웹 요소를 찾습니다.
        link1_elements = driver.find_elements(By.CLASS_NAME, 'link1')
        # 각 웹 요소의 href 값을 추출합니다.
        for link1_element in link1_elements:
            href = link1_element.get_attribute('href')
            s_tit_list.append(s_tit)
            href_list.append(href)
    except Exception as e:
        print(f"Error occurred at {i}th place: {e}")
# WebDriver 종료
driver.quit()
# href 값의 리스트를 데이터프레임으로 변환
df5 = pd.DataFrame({'s_tit': s_tit_list, 'link1_href': href_list})
# 데이터프레임 출력
print(df5)

100%|██████████| 1004/1004 [2:09:00<00:00,  7.71s/it] 


                                 s_tit  \
0                            나미송 머무는 곳   
1                           해비치호텔앤드리조트   
2                               신라호텔제주   
3                              롯데호텔 제주   
4     휴양펜션 스위스마을 Swiss Village Pension   
...                                ...   
999                              훼밀리민박   
1000                            휴마루리조트   
1001                              휴일기록   
1002                             히든스테이   
1003                            힐링캠프펜션   

                                             link1_href  
0     https://www.visitjeju.net/kr/detail/view?conte...  
1     https://www.visitjeju.net/kr/detail/view?conte...  
2     https://www.visitjeju.net/kr/detail/view?conte...  
3     https://www.visitjeju.net/kr/detail/view?conte...  
4     https://www.visitjeju.net/kr/detail/view?conte...  
...                                                 ...  
999   https://www.visitjeju.net/kr/detail/view?conte...  
1000  https://www.visitjeju.net

In [5]:
# DataFrame을 CSV 파일로 저장
df5.to_csv('sleep_url.csv', index=False)


# 세부정보 추출

In [24]:
import pandas as pd
df = pd.read_csv('food_url.csv')
# 칼럼명 변경
df = df.rename(columns={'s_tit': 'name', 'link1_href': 'url'})
url = df['url']

In [25]:
url

0       https://www.visitjeju.net/kr/detail/view?conte...
1       https://www.visitjeju.net/kr/detail/view?conte...
2       https://www.visitjeju.net/kr/detail/view?conte...
3       https://www.visitjeju.net/kr/detail/view?conte...
4       https://www.visitjeju.net/kr/detail/view?conte...
                              ...                        
1980    https://www.visitjeju.net/kr/detail/view?conte...
1981    https://www.visitjeju.net/kr/detail/view?conte...
1982    https://www.visitjeju.net/kr/detail/view?conte...
1983    https://www.visitjeju.net/kr/detail/view?conte...
1984    https://www.visitjeju.net/kr/detail/view?conte...
Name: url, Length: 1985, dtype: object

In [17]:
%pip install webdriver_manager

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [27]:
import json
import requests
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# 크롬 웹드라이버 불러오기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option("detach", True)
#path='/home/sadf/chromedriver'
driver = webdriver.Chrome(options=chrome_options)
#url = pd.read_csv('url.csv') # 여기에 실제 URL 목록을 넣어주세요.
# URL 목록
url_list =df['url']  

# 데이터를 담을 빈 리스트 생성
data_list = []

# URL 목록을 기반으로 반복문 실행
for url in tqdm(url_list[0:500]):

    # 각 URL에 접근하여 데이터 수집
    driver.get(url)
    driver.implicitly_wait(10)
    #2차 크롤링을 위한 bs4 셋팅
    session = requests.Session()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

    retries = Retry(total=5,
                    backoff_factor=0.1,
                    status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))  

    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    #지명
    name = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3')
    #별점
    score = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p')

    #태그
    tag = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area')

    #주소
    address = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont')

    #전화번호
    tel = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont')

    # 좋아요 개수
    like_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt')

    #찜하기 개수
    Steaming_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt')

    #리뷰 개수
    review_count =bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt')


    #방문했어요 수
    visit_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(5) > p.appraisal_cnt')

    #조회 수
    hits = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt')

    #sns 공유 수
    sns_share_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(7) > p.appraisal_cnt')


    detail_box_elements = bs.select_one('.add2020_detail_box_in')
    detail_box_side_elements = bs.select_one('.add2020_detail_side_info')
    #대표 사진 url
    # CSS 선택자를 사용하여 원하는 요소 찾기
    selected_element = bs.select_one('div.sub_visual_wrap')

    # image_url을 칼럼으로 만들어주기
    if selected_element:
        background_style = selected_element.get('style', '')
        if 'background' in background_style and 'url' in background_style:
            image_url = background_style.split("url(")[-1].split(")")[0].replace('&quot;', '')
    # 데이터를 딕셔너리에 저장 (비어 있는 경우 빈 문자열로 처리)
    data = {
        'Name': name.text.strip() if name else "",
        'Score': score.text.strip() if score else "",
        'Tag': tag.text.strip() if tag else "",
        'Address': address.text.strip() if address else "",
        'Tel': tel.text.strip() if tel else "",
        'Like Count': like_count.text.strip() if like_count else "",
        'Steaming Count': Steaming_count.text.strip() if Steaming_count else "",
        'Review Count': review_count.text.strip() if review_count else "",
        'Visit Count': visit_count.text.strip() if visit_count else "",
        'Hits': hits.text.strip() if hits else "",
        'SNS Share Count': sns_share_count.text.strip() if sns_share_count else "",
        'detail_box_elements': detail_box_elements.text.strip() if detail_box_elements else "",
        'detail_box_side_elements': detail_box_side_elements.text.strip() if detail_box_side_elements else "",
        'Image URL': image_url
    }
    # 데이터 리스트에 딕셔너리 추가
    data_list.append(data)
    time.sleep(2)
# 데이터 프레임 생성
df1 = pd.DataFrame(data_list)

# 데이터 csv 저장
df1.to_csv('foodinfo1.csv', index=False, encoding='utf-8')


100%|██████████| 500/500 [33:26<00:00,  4.01s/it]


In [28]:
import json
import requests
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# 크롬 웹드라이버 불러오기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option("detach", True)
#path='/home/sadf/chromedriver'
driver = webdriver.Chrome(options=chrome_options)
#url = pd.read_csv('url.csv') # 여기에 실제 URL 목록을 넣어주세요.
# URL 목록
url_list =df['url']  

# 데이터를 담을 빈 리스트 생성
data_list = []

# URL 목록을 기반으로 반복문 실행
for url in tqdm(url_list[500:1000]):

    # 각 URL에 접근하여 데이터 수집
    driver.get(url)
    driver.implicitly_wait(10)
    #2차 크롤링을 위한 bs4 셋팅
    session = requests.Session()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

    retries = Retry(total=5,
                    backoff_factor=0.1,
                    status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))  

    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    #지명
    name = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3')
    #별점
    score = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p')

    #태그
    tag = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area')

    #주소
    address = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont')

    #전화번호
    tel = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont')

    # 좋아요 개수
    like_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt')

    #찜하기 개수
    Steaming_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt')

    #리뷰 개수
    review_count =bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt')


    #방문했어요 수
    visit_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(5) > p.appraisal_cnt')

    #조회 수
    hits = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt')

    #sns 공유 수
    sns_share_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(7) > p.appraisal_cnt')


    detail_box_elements = bs.select_one('.add2020_detail_box_in')
    detail_box_side_elements = bs.select_one('.add2020_detail_side_info')
    #대표 사진 url
    # CSS 선택자를 사용하여 원하는 요소 찾기
    selected_element = bs.select_one('div.sub_visual_wrap')

    # image_url을 칼럼으로 만들어주기
    if selected_element:
        background_style = selected_element.get('style', '')
        if 'background' in background_style and 'url' in background_style:
            image_url = background_style.split("url(")[-1].split(")")[0].replace('&quot;', '')
    # 데이터를 딕셔너리에 저장 (비어 있는 경우 빈 문자열로 처리)
    data = {
        'Name': name.text.strip() if name else "",
        'Score': score.text.strip() if score else "",
        'Tag': tag.text.strip() if tag else "",
        'Address': address.text.strip() if address else "",
        'Tel': tel.text.strip() if tel else "",
        'Like Count': like_count.text.strip() if like_count else "",
        'Steaming Count': Steaming_count.text.strip() if Steaming_count else "",
        'Review Count': review_count.text.strip() if review_count else "",
        'Visit Count': visit_count.text.strip() if visit_count else "",
        'Hits': hits.text.strip() if hits else "",
        'SNS Share Count': sns_share_count.text.strip() if sns_share_count else "",
        'detail_box_elements': detail_box_elements.text.strip() if detail_box_elements else "",
        'detail_box_side_elements': detail_box_side_elements.text.strip() if detail_box_side_elements else "",
        'Image URL': image_url
    }
    # 데이터 리스트에 딕셔너리 추가
    data_list.append(data)
    time.sleep(2)
# 데이터 프레임 생성
df2 = pd.DataFrame(data_list)

# 데이터 csv 저장
df2.to_csv('foodinfo2.csv', index=False, encoding='utf-8')


100%|██████████| 500/500 [32:50<00:00,  3.94s/it]


In [29]:
import json
import requests
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# 크롬 웹드라이버 불러오기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option("detach", True)
#path='/home/sadf/chromedriver'
driver = webdriver.Chrome(options=chrome_options)
#url = pd.read_csv('url.csv') # 여기에 실제 URL 목록을 넣어주세요.
# URL 목록
url_list =df['url']  

# 데이터를 담을 빈 리스트 생성
data_list = []

# URL 목록을 기반으로 반복문 실행
for url in tqdm(url_list[1000:]):

    # 각 URL에 접근하여 데이터 수집
    driver.get(url)
    driver.implicitly_wait(10)
    #2차 크롤링을 위한 bs4 셋팅
    session = requests.Session()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

    retries = Retry(total=5,
                    backoff_factor=0.1,
                    status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))  

    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    #지명
    name = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3')
    #별점
    score = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.score_area_l > p')

    #태그
    tag = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.tag_area')

    #주소
    address = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont')

    #전화번호
    tel = bs.select_one('#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont')

    # 좋아요 개수
    like_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(1) > button > p.appraisal_cnt')

    #찜하기 개수
    Steaming_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(2) > button > p.appraisal_cnt')

    #리뷰 개수
    review_count =bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(3) > p.appraisal_cnt')


    #방문했어요 수
    visit_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(5) > p.appraisal_cnt')

    #조회 수
    hits = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt')

    #sns 공유 수
    sns_share_count = bs.select_one('#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(7) > p.appraisal_cnt')


    detail_box_elements = bs.select_one('.add2020_detail_box_in')
    detail_box_side_elements = bs.select_one('.add2020_detail_side_info')
    #대표 사진 url
    # CSS 선택자를 사용하여 원하는 요소 찾기
    selected_element = bs.select_one('div.sub_visual_wrap')

    # image_url을 칼럼으로 만들어주기
    if selected_element:
        background_style = selected_element.get('style', '')
        if 'background' in background_style and 'url' in background_style:
            image_url = background_style.split("url(")[-1].split(")")[0].replace('&quot;', '')
    # 데이터를 딕셔너리에 저장 (비어 있는 경우 빈 문자열로 처리)
    data = {
        'Name': name.text.strip() if name else "",
        'Score': score.text.strip() if score else "",
        'Tag': tag.text.strip() if tag else "",
        'Address': address.text.strip() if address else "",
        'Tel': tel.text.strip() if tel else "",
        'Like Count': like_count.text.strip() if like_count else "",
        'Steaming Count': Steaming_count.text.strip() if Steaming_count else "",
        'Review Count': review_count.text.strip() if review_count else "",
        'Visit Count': visit_count.text.strip() if visit_count else "",
        'Hits': hits.text.strip() if hits else "",
        'SNS Share Count': sns_share_count.text.strip() if sns_share_count else "",
        'detail_box_elements': detail_box_elements.text.strip() if detail_box_elements else "",
        'detail_box_side_elements': detail_box_side_elements.text.strip() if detail_box_side_elements else "",
        'Image URL': image_url
    }
    # 데이터 리스트에 딕셔너리 추가
    data_list.append(data)
    time.sleep(2)
# 데이터 프레임 생성
df3 = pd.DataFrame(data_list)

# 데이터 csv 저장
df3.to_csv('foodinfo3.csv', index=False, encoding='utf-8')


100%|██████████| 985/985 [1:04:36<00:00,  3.94s/it]


In [21]:
df2 = pd.read_csv('sleepinfo2.csv')
df3 = pd.read_csv('sleepinfo1.csv')

In [30]:
merged_df = pd.concat([df1, df2, df3], axis=0)

In [31]:
merged_df.to_csv('food_all_list.csv',index= False, encoding= 'utf-8')